In [1]:
import torch
from utils import *
from collections import defaultdict
import matplotlib.pyplot as plt
import time

from models.rendering import *
# from models.nerf import *

import metrics

from datasets import dataset_dict
from datasets.llff import *

torch.backends.cudnn.benchmark = True

img_wh = (320, 240)

# dataset = dataset_dict['llff'] \
#           ('/home/ubuntu/data/nerf_example_data/nerf_llff_data/fern/', 'test_train', spheric_poses=False,
#            img_wh=img_wh)

# dataset = dataset_dict['llff_nocs'] \
#           ('data/scene_1',
#            img_wh=img_wh)
dataset = dataset_dict['pd_multi_ae']

print("dataset", dataset)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
dataset datasets.pd_multi_ae.PD_Multi_AE


In [3]:
from torch.utils.data import DataLoader
from functools import partial
from datasets.pd_multi_ae import collate_lambda_train, collate_lambda_val
kwargs = {'root_dir': '/mnt/data/datasets/PDStep/v2/train/',
          'img_wh': tuple(img_wh),
          'white_back': True,
          'model_type': 'refnerf'}

train_dataset = dataset(split='val', **kwargs)

dataloader =  DataLoader(train_dataset,
                  shuffle=True,
                  num_workers=0,
                  batch_size=1,
                  #batch_size=1,
                  pin_memory=False,
                  #collate_fn = partial(collate_lambda_train, model_type='refnerf', ray_batch_size=2048)
                  collate_fn = partial(collate_lambda_val, model_type='refnerf')
                        )

In [ ]:
import torch
import numpy as np
a = torch.tensor([[0.6, 0.0, 0.0, 0.0],
[0.0, 0.4, 0.0, 0.0],
[0.0, 0.0, 1.2, 0.0],
[0.0, 0.0, 0.0,-0.4]])
b = torch.nonzero(a)

print(b.shape)
c = np.nonzero(a.numpy())
c = np.transpose(np.asarray(c))
print(np.transpose(np.asarray(c)).shape)
print(np.nonzero(a.numpy()))

print(np.equal(b.numpy(), c))

In [4]:
print("len train dataset", len(train_dataset))
count = 0
for i,data in enumerate(dataloader):
    print("i", i)
#     if i>0:
#         break
#     imgs, rgbs, rays, rays_d, view_dirs, instance_masks, radii, multloss, normals = data
    print("============================\n\n\n")
    for k,v in data.items():
        print(k,v.shape)
    print("========================\n\n\n")
    count+=1

print("count", count)
    
    #print("imgs, rgbs, rays", imgs.shape, rgbs.shape, rays.shape, instance_masks.shape, radii.shape)
#     for k,v in data.items():
#         print("k, v", k, v.shape)
#     count+=1
# print(count)
#     print("data_rgb", data['rgbs'].shape)

len train dataset 150
instance_dir suv_large_01_SWAT_01_black
rays torch.Size([49450, 3])
i 0



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([49450, 3])
rays_d torch.Size([49450, 3])
viewdirs torch.Size([49450, 3])
target torch.Size([49450, 3])
radii torch.Size([49450])
multloss torch.Size([3, 1])
normals torch.Size([49450, 3])
instance_mask torch.Size([49450, 1])
img_wh (2,)



instance_dir compact_mini_01_body_lightBlue
rays torch.Size([13156, 3])
i 1



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([13156, 3])
rays_d torch.Size([13156, 3])
viewdirs torch.Size([13156, 3])
target torch.Size([13156, 3])
radii torch.Size([13156])
multloss torch.Size([3, 1])
normals torch.Size([13156, 3])
instance_mask torch.Size([13156, 1])
img_wh (2,)



instance_dir suv_medium_01_body_green


/home/ubuntu/anaconda3/envs/nerf_pl/lib/python3.6/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


rays torch.Size([12915, 3])
i 2



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([12915, 3])
rays_d torch.Size([12915, 3])
viewdirs torch.Size([12915, 3])
target torch.Size([12915, 3])
radii torch.Size([12915])
multloss torch.Size([3, 1])
normals torch.Size([12915, 3])
instance_mask torch.Size([12915, 1])
img_wh (2,)



instance_dir midsize_sedan_02_body_blue
rays torch.Size([9680, 3])
i 3



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([9680, 3])
rays_d torch.Size([9680, 3])
viewdirs torch.Size([9680, 3])
target torch.Size([9680, 3])
radii torch.Size([9680])
multloss torch.Size([3, 1])
normals torch.Size([9680, 3])
instance_mask torch.Size([9680, 1])
img_wh (2,)



instance_dir compact_mini_02_blueMetallic
rays torch.Size([19317, 3])
i 4



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([19317, 3])
rays_d torch.Size([19317, 3])
viewdirs torch.Size([19317, 3])
target torch.Size([19317, 3])
radii torch.Size([19317])
multloss torch.Size([3, 1])
normals torch.Size([19

rays torch.Size([21280, 3])
i 25



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([21280, 3])
rays_d torch.Size([21280, 3])
viewdirs torch.Size([21280, 3])
target torch.Size([21280, 3])
radii torch.Size([21280])
multloss torch.Size([3, 1])
normals torch.Size([21280, 3])
instance_mask torch.Size([21280, 1])
img_wh (2,)



instance_dir compact_luxury_001_body_silver
rays torch.Size([25600, 3])
i 26



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([25600, 3])
rays_d torch.Size([25600, 3])
viewdirs torch.Size([25600, 3])
target torch.Size([25600, 3])
radii torch.Size([25600])
multloss torch.Size([3, 1])
normals torch.Size([25600, 3])
instance_mask torch.Size([25600, 1])
img_wh (2,)



instance_dir compact_sport_01_blue
rays torch.Size([21710, 3])
i 27



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([21710, 3])
rays_d torch.Size([21710, 3])
viewdirs torch.Size([21710, 3])
target torch.Size([21710, 3])
radii torch.Size([21710])
multloss torch.Size([3, 1])
normals torch.

rays torch.Size([20532, 3])
i 49



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([20532, 3])
rays_d torch.Size([20532, 3])
viewdirs torch.Size([20532, 3])
target torch.Size([20532, 3])
radii torch.Size([20532])
multloss torch.Size([3, 1])
normals torch.Size([20532, 3])
instance_mask torch.Size([20532, 1])
img_wh (2,)



instance_dir compact_mini_02_green
rays torch.Size([16352, 3])
i 50



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([16352, 3])
rays_d torch.Size([16352, 3])
viewdirs torch.Size([16352, 3])
target torch.Size([16352, 3])
radii torch.Size([16352])
multloss torch.Size([3, 1])
normals torch.Size([16352, 3])
instance_mask torch.Size([16352, 1])
img_wh (2,)



instance_dir suv_large_01_body_silver
rays torch.Size([30694, 3])
i 51



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([30694, 3])
rays_d torch.Size([30694, 3])
viewdirs torch.Size([30694, 3])
target torch.Size([30694, 3])
radii torch.Size([30694])
multloss torch.Size([3, 1])
normals torch.Size([

rays torch.Size([35340, 3])
i 73



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([35340, 3])
rays_d torch.Size([35340, 3])
viewdirs torch.Size([35340, 3])
target torch.Size([35340, 3])
radii torch.Size([35340])
multloss torch.Size([3, 1])
normals torch.Size([35340, 3])
instance_mask torch.Size([35340, 1])
img_wh (2,)



instance_dir suv_medium_02_silver
rays torch.Size([22781, 3])
i 74



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([22781, 3])
rays_d torch.Size([22781, 3])
viewdirs torch.Size([22781, 3])
target torch.Size([22781, 3])
radii torch.Size([22781])
multloss torch.Size([3, 1])
normals torch.Size([22781, 3])
instance_mask torch.Size([22781, 1])
img_wh (2,)



instance_dir midsize_sedan_03_body_beige
rays torch.Size([20592, 3])
i 75



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([20592, 3])
rays_d torch.Size([20592, 3])
viewdirs torch.Size([20592, 3])
target torch.Size([20592, 3])
radii torch.Size([20592])
multloss torch.Size([3, 1])
normals torch.Size

rays torch.Size([24640, 3])
i 96



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([24640, 3])
rays_d torch.Size([24640, 3])
viewdirs torch.Size([24640, 3])
target torch.Size([24640, 3])
radii torch.Size([24640])
multloss torch.Size([3, 1])
normals torch.Size([24640, 3])
instance_mask torch.Size([24640, 1])
img_wh (2,)



instance_dir compact_luxury_001_body_white
rays torch.Size([30132, 3])
i 97



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([30132, 3])
rays_d torch.Size([30132, 3])
viewdirs torch.Size([30132, 3])
target torch.Size([30132, 3])
radii torch.Size([30132])
multloss torch.Size([3, 1])
normals torch.Size([30132, 3])
instance_mask torch.Size([30132, 1])
img_wh (2,)



instance_dir fullsize_sedan_01_body_blue
rays torch.Size([46540, 3])
i 98



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([46540, 3])
rays_d torch.Size([46540, 3])
viewdirs torch.Size([46540, 3])
target torch.Size([46540, 3])
radii torch.Size([46540])
multloss torch.Size([3, 1])
normals t

rays torch.Size([31842, 3])
i 120



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([31842, 3])
rays_d torch.Size([31842, 3])
viewdirs torch.Size([31842, 3])
target torch.Size([31842, 3])
radii torch.Size([31842])
multloss torch.Size([3, 1])
normals torch.Size([31842, 3])
instance_mask torch.Size([31842, 1])
img_wh (2,)



instance_dir midsize_sedan_04_blue
rays torch.Size([39913, 3])
i 121



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([39913, 3])
rays_d torch.Size([39913, 3])
viewdirs torch.Size([39913, 3])
target torch.Size([39913, 3])
radii torch.Size([39913])
multloss torch.Size([3, 1])
normals torch.Size([39913, 3])
instance_mask torch.Size([39913, 1])
img_wh (2,)



instance_dir suv_medium_01_body_brown
rays torch.Size([35108, 3])
i 122



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([35108, 3])
rays_d torch.Size([35108, 3])
viewdirs torch.Size([35108, 3])
target torch.Size([35108, 3])
radii torch.Size([35108])
multloss torch.Size([3, 1])
normals torch.Siz

rays torch.Size([29329, 3])
i 144



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([29329, 3])
rays_d torch.Size([29329, 3])
viewdirs torch.Size([29329, 3])
target torch.Size([29329, 3])
radii torch.Size([29329])
multloss torch.Size([3, 1])
normals torch.Size([29329, 3])
instance_mask torch.Size([29329, 1])
img_wh (2,)



instance_dir suv_medium_01_trailerdefault
rays torch.Size([25648, 3])
i 145



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([25648, 3])
rays_d torch.Size([25648, 3])
viewdirs torch.Size([25648, 3])
target torch.Size([25648, 3])
radii torch.Size([25648])
multloss torch.Size([3, 1])
normals torch.Size([25648, 3])
instance_mask torch.Size([25648, 1])
img_wh (2,)



instance_dir midsize_sedan_03_body_silver
rays torch.Size([23635, 3])
i 146



src_imgs torch.Size([3, 128, 128])
rays_o torch.Size([23635, 3])
rays_d torch.Size([23635, 3])
viewdirs torch.Size([23635, 3])
target torch.Size([23635, 3])
radii torch.Size([23635])
multloss torch.Size([3, 1])
normal

In [ ]:

B, N, _ = data["rays_o"].shape

rays_o, rays_d, radii = data["rays_o"].reshape(-1,3), data["rays_d"].reshape(-1,3), data["radii"].reshape(-1,3)

print("rays_o", rays_o.shape)

In [ ]:
from models.resnet_encoder import MultiHeadImgEncoder
encoder = MultiHeadImgEncoder()

latents = encoder(data["src_imgs"])

print("latents", latents["density"].shape)
print("latents", latents["color"].shape)

In [ ]:
import matplotlib.pyplot as plt

print(data["img_wh"])

w,h = data["img_wh"]
print(data["src_imgs"].shape)
plt.imshow(data["src_imgs"].permute(1,2,0).numpy())
plt.show()

plt.imshow(data["target"].reshape(h,w,3).numpy())
plt.show()
    

In [ ]:
embedding_xyz = Embedding(10)
embedding_dir = Embedding(4)

nerf_coarse = NeRF()
nerf_fine = NeRF()

# ckpt_path = 'ckpts_old/fern/epoch=29.ckpt'
ckpt_path = 'ckpts_old/lego/epoch=15.ckpt'

load_ckpt(nerf_coarse, ckpt_path, model_name='nerf_coarse')
load_ckpt(nerf_fine, ckpt_path, model_name='nerf_fine')

nerf_coarse.cuda().eval()
nerf_fine.cuda().eval();

In [ ]:
models = {'coarse': nerf_coarse, 'fine': nerf_fine}
embeddings = {'xyz': embedding_xyz, 'dir': embedding_dir}

N_samples = 64
N_importance = 64
use_disp = False
chunk = 1024*32*4

@torch.no_grad()
def f(rays):
    """Do batched inference on rays using chunk."""
    B = rays.shape[0]
    results = defaultdict(list)
    for i in range(0, B, chunk):
        rendered_ray_chunks = \
            render_rays(models,
                        embeddings,
                        rays[i:i+chunk],
                        N_samples,
                        use_disp,
                        0,
                        0,
                        N_importance,
                        chunk,
                        dataset.white_back,
                        test_time=True)

        for k, v in rendered_ray_chunks.items():
            results[k] += [v]

    for k, v in results.items():
        results[k] = torch.cat(v, 0)
    return results

In [ ]:
sample = dataset[0]
rays = sample['rays'].cuda()

t = time.time()
results = f(rays)
torch.cuda.synchronize()
print(time.time()-t)

In [ ]:
img_gt = sample['rgbs'].view(img_wh[1], img_wh[0], 3)
img_pred = results['rgb_fine'].view(img_wh[1], img_wh[0], 3).cpu().numpy()
alpha_pred = results['opacity_fine'].view(img_wh[1], img_wh[0]).cpu().numpy()
depth_pred = results['depth_fine'].view(img_wh[1], img_wh[0])

print('PSNR', metrics.psnr(img_gt, img_pred).item())

plt.subplots(figsize=(15, 8))
plt.tight_layout()
plt.subplot(221)
plt.title('GT')
plt.imshow(img_gt)
plt.subplot(222)
plt.title('pred')
plt.imshow(img_pred)
plt.subplot(223)
plt.title('depth')
plt.imshow(visualize_depth(depth_pred).permute(1,2,0))
plt.show()